# Lengths and Dot Products

## Dot Product

In [2]:
import numpy as np 
vector_1 = np.array([1, 2])
vector_2 = np.array([4, 5])

print(vector_1.dot(vector_2))

14


## Vector Length

In [3]:
vector = np.array([1,3,2])

print(np.linalg.norm(vector))

3.7416573867739413


## Angle between two vectors


In [4]:
# https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2, degrees = True):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    radians = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    if degrees:
        return np.degrees(radians)
    else:
        return radians

v1 = np.array([2,2,-1])
v2 = np.array([2,-1,2])

print(angle_between(v1, v2))

90.0


# Matrices 

## Matrix multiplication

There's no built in type for vectors and matrices in python.  So I'm going to use numpy

In [5]:
vector = np.array([1, 2, 3, 4, 5, 6])

matrix_1 = np.array([[0, 1], [-9, 6]])

print(matrix_1 @ matrix_1)

[[ -9   6]
 [-54  27]]


## Solving Matrix Equations

In [163]:
import sympy as sp

# https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html
# Solve the system of equations x0 + 2 * x1 = 1 and 3 * x0 + 5 * x1 = 2:
a = np.array([[1, 2], [2, 5]])
b = np.array([1, 0])
print(np.linalg.solve(a, b))

# https://stackoverflow.com/questions/31547657/how-can-i-solve-system-of-linear-equations-in-sympy
a_2 = sp.Matrix(np.array([[1, 4, 7], [2, 5, 8], [3, 6, 9]]))
sp.linsolve(a_2)

[ 5. -2.]


{(-1, 2)}

# Elimination, A = LU factorization, Inverses

## With Numpy/Sympy

In [164]:
from scipy.linalg import lu
from numpy.linalg import inv
from sympy import Symbol, symbols


A = np.array([[.6, -1], [.4, 1]])

# LU with numpy
P, L, U = lu(A)
# print("A:", A)
# print("P:", P)
# print("L:", L)
# print("U:", U)

D = np.diag(np.diag(U))   # D is just the diagonal of U
U /= np.diag(U)[:, None]  # Normalize rows of U

# print("D:", D)
# print("U:", U)

# LU with sympy
A = sp.Matrix([[1,3,0],[2,4,0],[2,0,1]])
L, U, _ = A.LUdecomposition()
# pprint.pp(L)
# pprint.pp(U)
# pprint.pp(L.inv())

a,b,c,d = symbols('a,b,c,d')
A = sp.Matrix([[a,a,a,a],[a,b,b,b],[a,b,c,c],[a,b,c,d]])
L, U, _ = A.LUdecomposition()
pprint.pp(U)
pprint.pp(sp.simplify(U))


A_2 = np.float64(sp.Matrix([[1, 0, 0, 0],[1,1,0,0],[1,2,1,0],[1,3,3,1]]))
A_3 = np.float64(sp.Matrix([[1,0,0,0],[0,1,0,0],[0,1,1,0],[0,1,2,1]]))

# pprint.pp(A_3 @ inv(A_2))

A = sp.Matrix([[1, Symbol('a'), Symbol('b')], [0,1, Symbol('c')], [0,0,1]])
# pprint.pp(A.inv())


Matrix([
[a,      a,      a,      a],
[0, -a + b, -a + b, -a + b],
[0,      0, -b + c, -b + c],
[0,      0,      0, -c + d]])
Matrix([
[a,      a,      a,      a],
[0, -a + b, -a + b, -a + b],
[0,      0, -b + c, -b + c],
[0,      0,      0, -c + d]])


## My custom code

In [165]:
def identity_matrix(size):
    matrix = np.zeros([size, size])
    pivot_row = 0
    while (pivot_row < size):
        pivot_col = pivot_row
        matrix[pivot_row][pivot_col] = 1

        pivot_row += 1
    
    return matrix

def row_swap(matrix, row_a, row_b):
    temp = matrix[row_a].copy()
    matrix[row_a] = matrix[row_b]
    matrix[row_b] = temp
    return matrix

# Takes a matrix and returns it in upper triangular form, along with number of row swaps required
def forward_elimination(matrix):    
    num_rows, num_cols = matrix.shape
    
    pivot_row = 0
    pivot_col = 0

    row_swaps = 0

    while (pivot_row < num_rows):
        pivot = matrix[pivot_row][pivot_col]
        if (pivot == 0):
            found_swap = False
            for swap_pivot_row in range(pivot_row + 1, num_rows):
                swap_pivot = matrix[swap_pivot_row][pivot_col]
                if (swap_pivot != 0):
                    found_swap = True
                    row_swaps += 1
                    row_swap(matrix, pivot_row, swap_pivot_row)
                    pivot = matrix[pivot_row][pivot_col]
                    break
            if not found_swap:
                raise Exception('Could not find pivot or row swap for {}').format(pivot_row)
        # We have a non zero pivot.  Now let's subtract off the rows below.
        for following_row in range(pivot_row + 1, num_rows):
            to_eliminate = matrix[following_row][pivot_col]
            if (to_eliminate != 0):
                # Only need to worry about nonzero elements below
                l = to_eliminate / pivot
                for elim_col in range(pivot_col, num_cols):
                    matrix[following_row][elim_col] = matrix[following_row][elim_col] - (l * matrix[pivot_row][elim_col])
        pivot_row = pivot_row + 1
        pivot_col = pivot_col + 1
    
    return matrix, row_swaps

# Brings us from upper triangular (row echelon) to reduced row echelon form.
def rref(matrix):
    num_rows, num_cols = matrix.shape

    pivot_row = num_rows - 1

    # Loop from bottom to top
    while (pivot_row >= 0):
        pivot_col = pivot_row
        pivot = matrix[pivot_row][pivot_col]
        if pivot == 0:
            raise Exception('Unexpected zero on row {}').format(pivot_row)
        
        # Divide out the row if pivot is not already 1
        if pivot != 1:
            for divide_col in range(pivot_col, num_cols):
                matrix[pivot_row][divide_col] = matrix[pivot_row][divide_col] / pivot
            pivot = matrix[pivot_row][pivot_col]
        
        # Subtract off the rows above
        for previous_row in reversed(range(0, pivot_row)):
            to_eliminate = matrix[previous_row][pivot_col]
            if (to_eliminate != 0):
                # Only need to worry about nonzero elements above
                l = to_eliminate / pivot 
                for elim_col in range(pivot_col, num_cols):
                    matrix[previous_row][elim_col] = matrix[previous_row][elim_col] - (l * matrix[pivot_row][elim_col])
        
        pivot_row = pivot_row - 1
    
    return matrix

# Takes a matrix and finds its inverse with Gaussian elimination
def matrix_inverse(matrix):
    num_rows, num_cols = matrix.shape
    matrix = np.append(matrix, identity_matrix(num_rows), axis=1)
    matrix, row_swaps = forward_elimination(matrix)
    matrix = rref(matrix)

    # Return just the inverse
    return matrix[0:num_rows, num_cols:]

matrix = np.array([[.6, -1], [.4, 1]])
print(matrix_inverse(matrix))




[[ 1.   1. ]
 [-0.4  0.6]]


# Permutations


In [166]:
from itertools import permutations
from sympy import eye

# Generate a list of permutation matrices for a given size of matrix:


def generate_perm_matrices(size = 3):
    matrices = []

    M = eye(size)
    
    perms = permutations(range(size))

    # https://docs.sympy.org/latest/modules/matrices/common.html#sympy.matrices.common.MatrixCommon.permute
    for perm in perms:
        matrices.append(M.permute(perm))

    return matrices

matrices = generate_perm_matrices()

# Find the matrix that can be brought to power 3 and equal identity
for matrix in matrices:
    if (matrix != eye(3) and (matrix ** 3) == eye(3)):
        print(matrix)

print("4x4")
# Find the 4x4 matric that can be brought to power 4 and not equal identity
matrices = generate_perm_matrices(4)
print(len(matrices))
for matrix in matrices:
    if (matrix ** 4 != eye(4)):
        print(matrix)

Matrix([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
Matrix([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
4x4
24
Matrix([[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0]])
Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]])
Matrix([[0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1]])
Matrix([[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0]])
Matrix([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
Matrix([[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0]])
Matrix([[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0]])
Matrix([[0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0]])


# Determinants

## With numpy

In [167]:
a = np.array([[1, 2], [3, 4]])
print(np.linalg.det(a))

-2.0000000000000004



## Pivot Algorithm

Perform forward elimination to get $A$ to $U$.  If an odd number of row exchanges are involved, then multiply answer by $-1$. The answer being the product of the diagonal pivots.


In [168]:
def diagonal_product(matrix):
    num_rows = matrix.shape[0]

    product = 1

    pivot_row = 0
    pivot_col = 0

    while (pivot_row < num_rows):
        pivot = matrix[pivot_row][pivot_col]
        product = product * pivot

        pivot_row += 1
        pivot_col +=1
    
    return product

def determinant_by_pivots(matrix):
    num_rows, num_cols = matrix.shape
    if (num_rows != num_cols):
        raise Exception('Determinants are only for square matrices')

    matrix_u, row_swaps = forward_elimination(matrix)
    product = diagonal_product(matrix_u)

    if (row_swaps % 2 == 1):
        product = product * -1
    
    return product

matrix = np.array([[0, 0, 1], [0, 2, 3], [4, 5, 6]])
print(determinant_by_pivots(matrix))

-8


## The Big Formula for Determinants

For this one we split the determinant into selections from each column and row.  We can then extract the multipliers, and pull the sign from how many row swaps are required (even or odd) for the remaining permutation matrix.  We then add all those determinants together to get our determinant.


In [169]:
from itertools import permutations

# Takes a permutation matrix and returns its determinant based on number of swaps to reach identity matrix
def perm_determinant(permutation_matrix):
    num_rows, num_cols = permutation_matrix.shape

    pivot_row = 0
    pivot_col = 0

    row_swaps = 0

    while (pivot_row < num_rows):
        pivot = permutation_matrix[pivot_row][pivot_col]
        if pivot != 0 and pivot != 1:
            raise Exception("Only expecting 0s and 1s in a permutation matrix")

        if (pivot == 0):
            found_swap = False
            for swap_pivot_row in range(pivot_row + 1, num_rows):
                swap_pivot = permutation_matrix[swap_pivot_row][pivot_col]
                if swap_pivot != 0 and swap_pivot != 1:
                    raise Exception("Only expecting 0s and 1s in a permutation matrix")
                if (swap_pivot != 0):
                    found_swap = True
                    row_swaps += 1
                    row_swap(permutation_matrix, pivot_row, swap_pivot_row)
                    break
            if not found_swap:
                raise Exception('Could not find pivot or row swap for {}').format(pivot_row)
        
        pivot_row = pivot_row + 1
        pivot_col = pivot_col + 1
    
    if (row_swaps % 2 == 1):
        return -1
    else:
        return 1

def determinant_by_big_formula(matrix):
    num_rows, num_cols = matrix.shape
    perms = permutations(range(0,num_cols))

    determinant = 0

    for perm in perms:
        product = 1
        perm_matrix = np.zeros(matrix.shape)
        row = 0
        for col in perm:
            el = matrix[row][col]
            product = product * el
            perm_matrix[row][col] = 1
            row += 1
        product = product * perm_determinant(perm_matrix)
        determinant += product

    return determinant

print(determinant_by_big_formula(np.array([[0, 0, 1], [0, 2, 3], [4, 5, 6]])))

-8


# Transpose


In [170]:
# https://numpy.org/doc/stable/reference/generated/numpy.transpose.html

a = np.array([[1, 2], [3, 4]])
print(np.transpose(a))

[[1 3]
 [2 4]]


# Nullspace and column space

In [171]:
A = np.array([[1,2],[3,6]])
A_sym = sp.Matrix(A)

print(A_sym.nullspace())
print(A_sym.columnspace())

B_sym = sp.Matrix([[1,2,3]])
print(B_sym.nullspace())


[Matrix([
[-2],
[ 1]])]
[Matrix([
[1],
[3]])]
[Matrix([
[-2],
[ 1],
[ 0]]), Matrix([
[-3],
[ 0],
[ 1]])]


# (Reduced) Row Echelon Form

In [172]:
# https://stackoverflow.com/questions/70838696/how-do-i-find-the-row-echelon-form-ref
import pprint

Vec = sp.Matrix(np.matrix([[1,5,7,9],
                 [0,4,1,7],
                 [2,-2,11,-3]]))

echelon_form = Vec.echelon_form()

pprint.pp(echelon_form)

rref = Vec.rref()[0]

pprint.pp(rref)

Matrix([
[1, 5, 7, 9],
[0, 4, 1, 7],
[0, 0, 0, 0]])
Matrix([
[1, 0, 23/4, 1/4],
[0, 1,  1/4, 7/4],
[0, 0,    0,   0]])


# Rank of a Matrix

In [233]:
from sympy import solve

Vec = sp.Matrix(np.matrix([[1,5,7,9],
                 [0,4,1,7],
                 [2,-2,11,-3]]))

x, y = symbols("x, y")

print(Vec.rank())

print(sp.linsolve( (Vec, sp.Matrix([[0],[0],[0],[0]]))))

2
{(-23*tau0/4 - tau1/4, -tau0/4 - 7*tau1/4, tau0, tau1)}


# Checking independence of vectors

In [234]:
# Returns the rank of the matrix with the provided vectors as columns.  This tells you the maximum number of independent vectors.
def linearly_dependent(*vectors):
    return sp.Matrix.hstack(*vectors).rank()

v_1 = sp.Matrix([1,-1,0,0])
v_2 = sp.Matrix([1,0,-1,0])
v_3 = sp.Matrix([1,0,0,-1])
v_4 = sp.Matrix([0,1,-1,0])
v_5 = sp.Matrix([0,1,0,-1])
v_6 = sp.Matrix([0,0,1,-1])


print(linearly_dependent(v_1,v_2,v_3,v_4,v_5,v_6))

3


# Cross-Product

In [237]:
v_1 = sp.Matrix([2,1,0])
v_2 = sp.Matrix([-3, 0, 1])

print(v_1.cross(v_2))

Matrix([[1], [-2], [3]])




# Eigenvalues and Eigenvectors

In [238]:
# From https://pythonnumericalmethods.berkeley.edu/notebooks/chapter15.04-Eigenvalues-and-Eigenvectors-in-Python.html
import numpy as np
from numpy.linalg import eig 

a = np.array([[0, 2], 
              [2, 3]])
w,v=eig(a)
print('E-value:', w)
print('E-vector', v)

E-value: [-1.  4.]
E-vector [[-0.89442719 -0.4472136 ]
 [ 0.4472136  -0.89442719]]
